# Table of Contents

[Introduction](#Introduction)

[Functions to predict times](#Functions-to-predict-times)

[Evaluation Functions](#Evaluation-Functions)

[Weighted season prediction](#Weighted-season-prediction)

[Probability Distribution Predictions](#Probability-Distribution-Predictions)

[Model comparisons](#Model-comparisons)

[Complications](#Complications)

In [1]:
# First a cell to prepare the notebook for the stuff that I might need to use. More imports can be added as necessary.
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 
%load_ext memory_profiler

from fnmatch import fnmatch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pattern import web
import seaborn as sns
import math

# And the additional modules that I've used

import fnmatch
import os
import pickle
from PyPDF2 import PdfFileReader
from tabula import read_pdf
import urllib
import random
import sklearn
import scipy.stats as stats
import re

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import ElasticNet

import joblib

import matplotlib as mpl
sns.set(color_codes=True)

# Introduction

In this fourth notebook of the series, I want to take the weight combination that I found to be the best in the previous notebook, and I want to compare it to what I am calling the <i>dist draw model</i> or <i>exponentially modified normal model</i>. In particular, this  model assumes that all of a racer's times are distributed in an exponentially modified normal form, and that there are no external factors that would influence where in the distribution a racer's time would fall for a particular race. 

To begin with, I need to load in the some of the dataframes that were created in previous notebooks. In particular, I want the dataframes that contain speed, prone and standing accuracies, and prone and standing range times for all biathletes over all events. In addition, I want the dataframes containing quantitative snow and weather similarities as well as altitude similarities.

Previous Section: [Functions to predict times](#Functions-to-predict-times)

[Table of Contents](#Table-of-Contents)



In [2]:
absolute_mens_speed = pd.read_pickle('absolute_mens_speed.pkl')
absolute_mens_prone_range = pd.read_pickle('absolute_mens_prone_range.pkl')
absolute_mens_prone_shooting = pd.read_pickle('absolute_mens_prone_shooting.pkl')
absolute_mens_standing_range = pd.read_pickle('absolute_mens_standing_range.pkl')
absolute_mens_standing_shooting = pd.read_pickle('absolute_mens_standing_shooting.pkl')


In [3]:
wc_quant_weather_similarities = pd.read_pickle('wc_quant_weather_similarities.pkl')
ibu_quant_weather_similarities = pd.read_pickle('ibu_quant_weather_similarities.pkl')

wc_quant_snow_similarities = pd.read_pickle('wc_quant_snow_similarities.pkl')
ibu_quant_snow_similarities = pd.read_pickle('ibu_quant_snow_similarities.pkl')

wc_altitude_similarities = pd.read_pickle('wc_altitude_similarities.pkl')
ibu_altitude_similarities = pd.read_pickle('ibu_altitude_similarities.pkl')

# Functions to predict times

In order to have models to compare, we need to make predictions. Because these functions are taken directly from the previous notebook (Biathlon Report C-a), I'm going to write fairly minimally here. The five functions found below are:
1. ```adjust_times```: It appears from looking at the data for individual racers that racer speeds generally increase somewhat linearly over time. The function adjust_times allows early career race speeds to be adjusted based on a linear fit of racer speed over all previous seasons (performed on a season by season basis) in order to try to mitigate the effects of time on speeds.
2. ```build_racer_speed_distribution```: Creates a list of racer speeds by taking all of a given racer's speeds and then repeating them with a multiplicity that is dependent on the similarities between the conditions for the current competition and the individual prior competitions. A prior race with a similarity value of 1 (all predictor variables under consideration have identical or nearly identical values) would have its speed appear 10 times in the list. A prior race with a similarity value of 0.5 (the distance between the predictor variables under consideration is roughly half of the total spread for that variable) would have its speed appear 5 times on the list. ```build_racer_speed_distribution``` calls ```adjust_times``` to adjust the speeds for events that were held before the season under consideration to reflect general improvements in speed.
3. ```build_racer_pr_distribution```:  Creates a list of n predicted total range times for a given racer in a given event. This is a fairly complicated process that involves the following steps:
    1. For the given racer, determine which previous races (of both types) that racer has competed in. Collect range and accuracy (either prone or standing, depending on circumstance) into a pair of lists.
    2. Using the weights associated to range times, produce a pair of weighted lists for range times and accuracy. Take a paired bootstrap sample (use the same ordered list of indices for both lists) and perform a linear regression on the results. The intercept is taken as the shooting time, and the slope is taken as the penalty loop time for any missed shots.
    3. Using the weights associated to accuracy, produce a weighted list for accuracy. The mean of this list (or rather $a = (5-\bar{x})/5$) will be taken to be the expected probability of making a particular shot. Five random values in the interval [0,1] are then computed. For each value, if the value is below $a$, the shot is made. Otherwise, the shot is missed.
    4. The total range time is calculated as the sum of the shooting time and the product of the number of missed shots and the penalty loop time.
4. ```racer_time_predict```: For a given racer, this function 
    1. first calls ```racer_speed_distribution```. Then, for each of the n desired predictions, it randomly selects a sample of size 10 from the returned list and computes the average.
    2. next calls ```build_racer_pr_distribution``` twice, once for prone range times and once for standing range times. 
    3. finally, adds together the times in the resulting lists to produce a single list of n predicted race times for the given biathlete in the given race
5. ```race_time_predictions```: calls ```racer_time_predict``` for each of the racers competing in a given race and stitches the outcomes together to form a single dataframe.

Previous Section: [Introduction](#Introduction)

Next Section: [Evaluation Functions](#Evaluation-Functions)



[Table of Contents](#Table-of-Contents)



In [4]:
"""
Function
--------

adjust_times : Takes a racer and his event speeds over the course of a career, finds the
               best fit line through the data, and adjusts early speeds to reflect what 
               they would be predicted to be if the race were run in the season under 
               consideration.

Parameters
----------

racer : a string containing the name of a biathlete
season : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.

Returns
-------

adjusted_racer : a list containing the racer's speed adjusted for the season under 
                 consideration

Examples
--------
"""

# To be called inside speed_predictions

def adjust_times(racer, season):
    
    indices = racer.index.tolist()
    years = [item.split(':')[1] for item in indices]
    years = [''.join(['20',item[2:4]]) for item in years]
    years = [float(item) for item in years]
    years = np.array(years).reshape(-1,1)
    speeds = np.array(racer.tolist()).reshape(-1,1)
    
    linreg = LinearRegression()
    linreg.fit(years , speeds)
    
    coef = linreg.coef_[0][0]
    
    #print coef
    # Now that we know the slope of the best fit line here, I want to create a 
    # time adjusted version of the speeds
    
    adjusted_racer = racer.copy()
    
    for i in range(len(racer)):
        time_delta = float(season)- years[i][0]
        #print time_delta
        adjusted_racer[i] = adjusted_racer[i] + coef*time_delta
        
    return adjusted_racer

In [5]:
"""
Function
--------

build_racer_speed_distribution : takes a racer, season, event, and dataframes containing
                                 similarity data about world cup vs world cup and world cup
                                 vs ibu cup race conditions, and returns a list of speeds
                                 from previous races, where the multiplicity of the speed
                                 is determined by the degree of similarity between the race
                                 of interest and the race from which the speed was taken

Parameters
----------

racer : a string containing the name of a biathlete
season : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.
event : a string that codes the event under consideration. Possible values are 'CP01', 
        'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08', 'CP09', 'CH__', 'OG__'
wc_sim : a dataframe containing values between 0 and 1 which codes the degree of similarity 
         between world cup (wc) races in a pairwise fashion. Pairs with a similarity value
         of 1 were run under nearly identical conditions, while those with a similarity
         value of 0 were run under extremely different conditions
ibu_sim : a dataframe containing values between 0 and 1 which codes the degree of similarity 
          between world cup (wc) races and ibu cup races in a pairwise fashion. 

Returns
-------

racer_predict : a list of speeds derived from the speeds of the racer's previous events. Each 
                prior speed is in the list with multiplicity n, where n is the rounded
                product of 10 and the similarity score for the pairing between the current
                event and the prior event

Examples
--------
"""

def build_racer_speed_distribution(racer, season, event, wc_sim, ibu_sim):

    col_name = ':'.join(['wc',season, event])
    racer_data = absolute_mens_speed.loc[racer, :col_name]
    name = racer
    actual = float(racer_data[col_name])
    short_racer_data = racer_data[2:-1].copy()
    short_racer_data.dropna(inplace = True)
    
    # To drop later
    #short_racer_data = short_racer_data[-20:]
    
    predictors = len(short_racer_data)
    
    year = ''.join(['20',season[2:4]])
    short_racer_data = adjust_times(short_racer_data, year)
    indices = short_racer_data.index.tolist()

    race_weights = []
    for index in indices:
        split_index = index.split(':')
        try:
            if split_index[0] == 'wc':
                race_weights.append(wc_sim.loc[':'.join([season,event]),
                                               ':'.join([split_index[1],split_index[2]])])
            else:
                race_weights.append(ibu_sim.loc[':'.join([season,event]),
                                                ':'.join([split_index[1],split_index[2]])])
        except:
            race_weights.append(0.0)
    race_weights_rounded = [int(round(item,1)*10) for item in race_weights]

    race_weights_rounded

    # Making the weighted data list
    
    racer_predict = []

    for i in range(len(short_racer_data)): 
        for j in range(race_weights_rounded[i]):
            racer_predict.append(float(short_racer_data[i]))

    return racer_predict

In [6]:
"""
Function
--------

build_racer_pr_distribution : takes a racer, season, event, and dataframes containing
                              similarity data about world cup vs world cup and world cup
                              vs ibu cup race conditions, and returns a list of predicted
                              range times for the competition under consideration

Parameters
----------

racer : a string containing the name of a biathlete
season : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.
event : a string that codes the event under consideration. Possible values are 'CP01', 
        'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08', 'CP09', 'CH__', 'OG__'
wc_acc_sim : a dataframe containing values between 0 and 1 which codes the degree of 
             similarity between world cup (wc) races in a pairwise fashion. Pairs with 
             a similarity value of 1 were run under nearly identical conditions, while
             those with a similarity value of 0 were run under extremely different 
             conditions. Chosen for predictive power for shooting accuracy
ibu_acc_sim : a dataframe containing values between 0 and 1 which codes the degree of 
              similarity between world cup (wc) races and ibu cup races in a pairwise 
              fashion. Chosen for predictive power for shooting accuracy
wc_range_sim : a dataframe containing values between 0 and 1 which codes the degree of 
               similarity between world cup (wc) races in a pairwise fashion. Chosen for
               predictive power for range time (shooting time together with penalty time)
ibu_range_sim : a dataframe containing values between 0 and 1 which codes the degree of 
                similarity between world cup (wc) races and ibu cup races in a pairwise 
                fashion. Chosen for predictive power for range time
n : the length of the list of predicted range times that is returned

Returns
-------

racer_predict : a list of predicted range times

Examples
--------
"""

def build_racer_pr_distribution(racer, season, event, wc_acc_sim, ibu_acc_sim,
                                wc_range_sim, ibu_range_sim,n):

    col_name = ':'.join(['wc',season, event])
    racer_time_data = absolute_mens_prone_range.loc[racer, :col_name]
    racer_shot_data = absolute_mens_prone_shooting.loc[racer, :col_name]
    name = racer
    actual = float(racer_time_data[col_name])
    short_racer_time_data = racer_time_data[2:-1].copy()
    short_racer_shot_data = racer_shot_data[2:-1].copy()
    short_racer_time_data.dropna(inplace = True)
    short_racer_shot_data.dropna(inplace = True)
    predictors = len(short_racer_shot_data)
    
    # To drop later
    
    short_racer_time_data = short_racer_time_data[-20:]
    short_racer_shot_data = short_racer_shot_data[-20:]

    
    year = ''.join(['20',season[2:4]])
    indices = short_racer_shot_data.index.tolist()
    
    # Separate weights for shooting accuracy and range times
    
    # Accuracy
    accuracy_weights = []
    for index in indices:
        split_index = index.split(':')
        if split_index[0] == 'wc':
            accuracy_weights.append(wc_acc_sim.loc[':'.join([season,event]),
                                                   ':'.join([split_index[1],split_index[2]])])
        else:
            try:
                accuracy_weights.append(ibu_acc_sim.loc[':'.join([season,event]),
                                                    ':'.join([split_index[1],split_index[2]])])
            except: # There is missing data
                accuracy_weights.append(0.0)
    accuracy_weights_rounded = [int(round(item,1)*10) for item in accuracy_weights]

    # Range times
    range_weights = []
    for index in indices:
        split_index = index.split(':')
        if split_index[0] == 'wc':
            range_weights.append(wc_range_sim.loc[':'.join([season,event]),
                                                  ':'.join([split_index[1],split_index[2]])])
        else:
            try:
                range_weights.append(ibu_range_sim.loc[':'.join([season,event]),
                                                  ':'.join([split_index[1],split_index[2]])])
            except: # There is missing data
                range_weights.append(0.0)
    range_weights_rounded = [int(round(item,1)*10) for item in range_weights]

    # And I'm going to have to split now to build my distributions
    
        # Making the weighted data list
    
    racer_accuracy = []
    racer_shot = []
    racer_time = []
    racer_predict = []

    for i in range(len(short_racer_shot_data)): 
        for j in range(accuracy_weights_rounded[i]):
            racer_accuracy.append(float(short_racer_shot_data[i]))
            
    for i in range(len(short_racer_time_data)):
        for j in range(range_weights_rounded[i]):
            racer_shot.append(float(short_racer_shot_data[i]))
            racer_time.append(float(short_racer_time_data[i]))
            
    # Build a model for shooting time and penalty loop time
    
    for k in range(n): 
        index_sample = np.random.choice(range(len(racer_shot)), 
                                                len(racer_shot), replace = True)
        racer_shot_sample = [racer_shot[i] for i in index_sample]
        racer_time_sample = [racer_time[i] for i in index_sample]
        
        accuracy = np.mean(racer_accuracy)/5
        
        linreg = LinearRegression()
        linreg.fit(np.array(racer_shot_sample).reshape(-1,1),
                                       np.array(racer_time_sample).reshape(-1,1))
        loop = linreg.coef_
        shot_time = linreg.intercept_
        
    # And predict number of missed shots

        shooting = np.random.sample(5)
        count = 0
        for m in range(5):
            if shooting[m] < accuracy:
                count += 1
        range_time = shot_time + count*loop
        racer_predict.append(range_time[0][0])
          
    return racer_predict

In [7]:
"""
Function
--------

racer_time_predict : produces a list of predicted times for a given racer in a given 
                     competition

Parameters
----------

racer : a string containing the name of a biathlete
year : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.
event : a string that codes the event under consideration. Possible values are 'CP01', 
        'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08', 'CP09', 'CH__', 'OG__'
n : an integer giving the desired number of times predicted
length : a float giving the length of the ski portion of the course for the competition
         under consideration
weight_type : a list of lists containing the world cup and ibu similarity weightings
              for the speed prediction and prone and standing range predictions

Returns
-------

race_predictions : an n element list containing predicted total times for the given
                   racer in the given competition

Examples
--------
"""



def racer_time_predict(racer,year,event,n,length, weight_type):
    
    # First up, produce the speed estimates
    
    weightings = weight_type
    #print weights[weightings[0]][0]
    #print weights[weightings[0]][1]
    
    speed_distribution = build_racer_speed_distribution(racer, year, event, 
                                    weights[weightings[0]][0], weights[weightings[0]][1])
    
    ski_time_predictions = []
    for i in range(n):
        speed_sample = np.random.choice(speed_distribution, 10)
        ski_time_predictions.append(length/np.mean(speed_sample))
        
    prone_range = build_racer_pr_distribution(racer, year, event, weights[weightings[1]][0], 
                                    weights[weightings[1]][1],weights[weightings[3]][0], 
                                              weights[weightings[3]][1],n)
    standing_range = build_racer_pr_distribution(racer,year,event, weights[weightings[2]][0],
                                    weights[weightings[2]][1],weights[weightings[4]][0],
                                                 weights[weightings[4]][1],n)
    
    time_predictions = [x+y+z for x,y,z in zip(ski_time_predictions, 
                                               prone_range, standing_range)]
    
    race_predictions = [racer]
    race_predictions.extend(time_predictions)
    
    return race_predictions

In [8]:
"""
Function
--------

race_time_predictions : calls racer_time_predict repeatedly to create a dataframe of 
                        time predictions for all of the racers in a given competition

Parameters
----------

year : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.
event : a string that codes the event under consideration. Possible values are 'CP01', 
        'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08', 'CP09', 'CH__', 'OG__'
n : an integer indicating the total number of time predictions to be made for each racer
weight_type : a list of lists containing the world cup and ibu similarity weightings
              for the speed prediction and prone and standing range predictions

Returns
-------

predicted_times : a dataframe containing whose index consists of those racers who competed
                  in the given race, and whose rows are the n predicted times for those
                  racers
problem_racers : a list of racers for whom racer_time_predict was unable to be executed,
                 generally due to lack of prior race data

Examples
--------
"""



def race_time_predictions(year,event,n, weight_type):
    
    # Find the length of the race
    course_url = 'course_summary_%(year)s_M.pkl' %{'year' : year}
    course_data = pd.read_pickle(course_url)
    length = course_data.loc[course_data['Event'] == event]['Length'].tolist()[0]
    
 #   print 'pickle data read'
 #   %memit
    
    # Get the list of racers
    
    race_code = ':'.join(['wc', year, event])
    racer_indices = absolute_mens_speed[race_code].dropna().index.tolist()

    # Make the predictions
    
    predicted_times = []
    problem_racers = []
    
    for racer in racer_indices[:-1]:
        try:
            predicted_times.append(racer_time_predict(racer, year, event ,n, length, 
                                                      weight_type))
        except:
            problem_racers.append(racer)
    
    predicted_times = pd.DataFrame(predicted_times)
    
    predicted_times.set_index(0, drop = True, inplace = True)
    
    return predicted_times, problem_racers

# Evaluation Functions

In order to evaluate the outcomes produced by our models, we need a number of functions that allow us to compare outcomes across multiple models. In order to do this, we define several additional functions.
1. ```place_counts```: The function ```race_time_predictions``` produces a dataframe containing one row for each competitor in a competition (with the exception of those racers who have at most a single prior race) and with $n$ columns of predicted times. The function ```place_counts``` treats each column as a running of the race, and determines, for each column, the order of finish predicted for each racer. It then aggregates this data to produce a dataframe which again has a row for each competitor, but whose columns contain the number of predicted first place finishes, second place finishes, third place finishes, etc.  
2. ```finding_percentiles```: The function ```race_time_predictions``` produces a dataframe containing one row for each competitor in a competition (with the exception of those racers who have at most a single prior race) and with $n$ columns of predicted times. The function ```finding_percentiles``` calculates, for each individual racer, various attributes of the distribution of times predicted for that racer, among them mean, median, 25th and 75th percentiles, and the difference between the racer's actual time and the mean of their predicted times. It then returns a dataframe which contains one row for each racer and columns for each attribute of their time distributions.
3. ```evaluating_percentiles```: This function takes the dataframe that is output by ```finding_percentiles``` and, for each racer, determines where in that racers distribution of time predictions their actual time falls. The results are then returned as a two column dataframe containing the racers' names in the first column and the code for the location of their actual times within their distributions as the second. The codes for the different parts of the distribution range are as follows:
    - 0 : actual time is faster than the minimum predicted time
    - 1 : actual time is between the minimum and the 5th percentile of the predicted times
    - 2 : actual time is between the 5th percentile and the 10th percentile of the predicted times
    - 3 : actual time is between the 10th percentile and the 25th percentile of the predicted times
    - 4 : actual time is between the 25th percentile and the median of the predicted times
    - 5 : actual time is between the median and the 75th percentile of the predicted times
    - 6 : actual time is between the 75th percentile and the 90th percentile of the predicted times
    - 7 : actual time is between the 90th percentile and the 95th percentile of the predicted times
    - 8 : actual time is between the 95th percentile and the maximum of the predicted times
    - 9 : actual time is slower than the maximum predicted time
4. ```evaluating_place_counts```: This function takes the dataframe that is output by ```place_counts``` and, for each racer, determines with what frequency the predicted place is correct, within one place of being correct, within two places of being correct, within five places of being correct, within ten places of being correct, and within twenty places of being correct. (For example, a racer who actually finished 35th would have all predicted finishes between 30th and 40th counted when determining the value for beining within five places of being correct.) It then returns a dataframe with one row for each competitor and one column for each of the seven categories under consideration.
5. ```evaluating_dist_from_mean```: This function takes the dataframe output by ```finding_percentiles```. It uses the column ```dist_from_mean``` to determine for what percentage of the biathletes the mean of their distributions were within 10 seconds of their actual times, within 25 seconds of their actual times, within 50 seconds of their actual times, within 100 seconds of their actual times, within 150 seconds of their actual times, and within 200 seconds of their actual times. The result is then returned as a dataframe with only a single column. For reference, a change of 10 seconds in finishing time corresponds to a difference (on average) of 4-5 places in finishing place, while a change in finishing time of 100 seconds corresponds to a difference (on average) of 36-37 places in finishing place.
6. ```average_from_mean```: This function takes the ```diff from mean``` column from the dataframe output by ```finding_percentiles``` and calculates both the mean of the values in the column and the mean of the absolute values of the entries in the column and returns both values as floats. The first computation gives us some indication of how balanced our errors in prediction are with respect to the actual times, since positive and negative values will cancel each other out. The second computation gives us an indication of overall error. In the case that the absolute values of these two numbers are the same (or nearly the same), it is an indication that the values being predicted tend to fall consistantly too high or too low.
7. ```check_predictions```: This function takes the dataframe produced by ```finding_percentiles``` and returns a dataframe that, for each racer, indicates (via ```True``` or ```False```) whether that racer's actual time fell in the middle 50% of his predicted time distribution and whether his actual time fell in the middle 90% of his predicted time distribution.
8. ```inside_outside```: This function takes the dataframe produced by ```check_predictions``` and returns two floats. The first is the percentage of the racers for whom the actual time fell inside of the middle 50% of their distribution, and the second is the percentage of the racers for whom the actual time fell inside of the middle 90% of their distribution.

Previous Section: [Functions to predict times](#Functions-to-predict-times)

Next Section: [Weighted season prediction](#Weighted-season-prediction)


[Table of Contents](#Table-of-Contents)



In [9]:
"""
Function
--------

place_counts : treats each column of a race_time_predictions dataframe as an instance
               of a race, and determines the finishing places for each racer within 
               that race

Parameters
----------

times : a dataframe that is the output of a call to race_time_predictions

Returns
-------

place_count : a dataframe of integers indicating which place a given racer would have 
              had if the kth column of 

Examples
--------
"""



def place_counts(times):
    
    predicted_places = times.copy()

    for j in range(predicted_places.shape[1]):
        predicted_places.sort_values(j+1,inplace = True)
        for i in range(len(predicted_places)):
            predicted_places.iloc[i,j] = i+1

    place_count = pd.DataFrame(columns = range(1, len(times)+1), index = times.index)
    
    for racer in place_count.index:
        for i in place_count.columns:
            count = len([item for item in predicted_places.loc[racer] if item == i])
            place_count.loc[racer,i] = count
            
    return place_count

In [10]:
"""
Function
--------

finding_percentiles : takes the output of a call to race_time_predictions and returns a
                      dataframe containing information about the distribution of predicted
                      times for each racer

Parameters
----------

times : a dataframe that is the output of a call to race_time_predictions
year : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.
event : a string that codes the event under consideration. Possible values are 'CP01', 
        'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08', 'CP09', 'CH__', 'OG__'

Returns
-------

racer_percentiles : a dataframe containing the name, mean time, standard deviation of time,
                    minimum time, fifth percentile, tenth percentile, twenty-fifth percentile,
                    median, seventy-fifth percentile, ninetieth percentile, ninety-fifth
                    percentile, maximum time, actual time, and difference between the actual
                    time and the mean predicted time for each racer

Examples
--------
"""



def finding_percentiles(times, year, event):
    
    racer_percentiles = []
    
    filename = 'companal_SMSP_%(year)s_%(event)s.pkl' %{'year': year, 'event' : event}
    event_data = pd.read_pickle(filename)
    
    for i in range(len(times)):
        name = times.index[i]
        racer = [name]
        racer_data = times.iloc[i,:]
        mean = np.mean(racer_data)
        stdev = np.std(racer_data)
        minimum = min(racer_data)
        per5 = np.percentile(racer_data,5)
        per10 = np.percentile(racer_data, 10)
        per25 = np.percentile(racer_data,25)
        median = np.median(racer_data)
        per75 = np.percentile(racer_data,75)
        per90 = np.percentile(racer_data,90)
        per95 = np.percentile(racer_data,95)
        maximum = max(racer_data)
        actual = event_data.loc[event_data['Name'] == name]['Total Time'].tolist()[0]
        difference = actual - mean
        racer.extend([mean, stdev, minimum, per5, per10, per25, median, per75, per90, per95,
                      maximum, actual, difference])
        racer_percentiles.append(racer)
        
    racer_percentiles = pd.DataFrame(racer_percentiles, columns = ['Name', 'mean', 'deviation',
                                                    'min', '5th per', '10th per', '25th per', 
                                                    'median', '75th per', '90th per', 
                                                    '95th per', 'maximum', 'actual time', 
                                                    'diff from mean'])
    return racer_percentiles

In [11]:
"""
Function
--------

evaluating_percentiles : takes a dataframe that is the output of a call to finding 
                         percentiles and returns a dataframe that indicates for each 
                         racer into what part of that racer's predicted times distribution
                         his actual time falls

Parameters
----------

percentiles : a dataframe that is the output of a call to finding_percentiles

Returns
-------

evaluation : a dataframe containing, for each racer, a code indicating in which portion 
             of that racer's predicted distribution his actual time falls.

Examples
--------
"""



def evaluating_percentiles(percentiles):
    
    evaluation = []
    for i in range(len(percentiles)):
        racer_data = percentiles.iloc[i,:]
        name = racer_data[0]
        actual = racer_data[-2]
        if actual < percentiles.iloc[i,3]:
            loc = 0
        elif percentiles.iloc[i,3] <= actual < percentiles.iloc[i,4]:
            loc = 1
        elif percentiles.iloc[i,4] <= actual < percentiles.iloc[i,5]:
            loc = 2
        elif percentiles.iloc[i,5] <= actual < percentiles.iloc[i,6]:
            loc = 3
        elif percentiles.iloc[i,6] <= actual < percentiles.iloc[i,7]:
            loc = 4
        elif percentiles.iloc[i,7] <= actual < percentiles.iloc[i,8]:
            loc = 5
        elif percentiles.iloc[i,8] <= actual < percentiles.iloc[i,9]:
            loc = 6
        elif percentiles.iloc[i,9] <= actual < percentiles.iloc[i,10]:
            loc = 7
        elif percentiles.iloc[i,10] <= actual < percentiles.iloc[i,11]:
            loc = 8
        else:
            loc = 9

        evaluation.append([name,loc])
        
    evaluation = pd.DataFrame(evaluation, columns = ['Name','Location'])
    
    return evaluation

In [12]:
"""
Function
--------

evaluating_place_counts : takes a dataframe that is the output of place_counts and, for each
                          racer, determines how many of the trial races had the racer placed
                          correctly, within one place of his actual finish place, within two 
                          places of his actual finish place, etc 

Parameters
----------

place_data : a dataframe that is the output of place_counts
year : a string that codes the season under consideration. It is of the form y1y2 where
       y1 is the last two digits of the year in which the season started, and y2 is the 
       last two digits of the year in which the season ended.
event : a string that codes the event under consideration. Possible values are 'CP01', 
        'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08', 'CP09', 'CH__', 'OG__'

Returns
-------

place_evaluations : a dataframe containing a row for each racer in the competition. Each
                    row in turn contains the number of predicted finishes places that were
                    correct, within one place of the true place,within two places of the 
                    true place,within three places of the true place, within five places 
                    of the true place, within ten places of the true place, and within
                    twenty places of the true place.

Examples
--------
"""



def evaluating_place_counts(place_data,year,event):
    
    # Get the actual places
    filename = 'companal_SMSP_%(year)s_%(event)s.pkl' %{'year' : year, 'event' : event}
    finish_place = pd.read_pickle(filename)[['Name','Total Time']]
    finish_place.set_index('Name', inplace = True, drop = True)
    finish_place = finish_place.loc[place_data.index]
    finish_place.sort_values('Total Time', inplace = True)
    finish_place.reset_index(inplace = True)
    finish_place.reset_index(inplace = True)
    finish_place.columns = ['Place','Name','Total Time']
    finish_place.set_index('Name', inplace = True, drop = True)
    
    place_evaluations = []
    
    for racer in place_data.index.tolist():
        racer_places = place_data.loc[racer]
        racer_actual = finish_place.loc[racer,'Place']+1
        racer_evaluation = [racer, racer_actual]
        
        for i in [0, 1, 2, 3, 5, 10, 20]:
            
            racer_range = range(racer_actual- i, racer_actual+i+1)
            possible_range = set(range(1,len(racer_places)+1))
            check_range = set(possible_range.intersection(racer_range))
            count = 0
            for j in check_range:
                count = count + racer_places[j]
            racer_evaluation.append(count)
            
        place_evaluations.append(racer_evaluation)
        
    place_evaluations = pd.DataFrame(place_evaluations)
    place_evaluations.columns = ['Name','Place','Correct','Within 1','Within 2','Within 3',
                                'Within 5','Within 10','Within 20']
    
    place_evaluations.sort_values('Place',inplace = True)
    return place_evaluations

In [13]:

def evaluating_dist_from_mean(diff_from_mean):
    
    evaluated_distances = []
    
    for i in [10,25,50,100,150,200]:
        count = 0
        for j in range(len(diff_from_mean)):
            if abs(diff_from_mean[j])<= i:
                count += 1
        evaluated_distances.append(float(count)/len(diff_from_mean)*100)
        
    evaluated_distances = pd.DataFrame(evaluated_distances, index = ['in 10', 'in 25',
                                                        'in 50', 'in 100', 'in 150','in 200'])
    
    return evaluated_distances

In [14]:
absolute_mens_time = pd.read_pickle('absolute_mens_time.pkl')

In [15]:
weights = {"quant_weather" : [wc_quant_weather_similarities, ibu_quant_weather_similarities],
           "quant_snow" : [wc_quant_snow_similarities, ibu_quant_snow_similarities],
           "altitude" : [wc_altitude_similarities, ibu_altitude_similarities]}

weight_combos = [['quant_snow','quant_weather','altitude','quant_snow','quant_weather']]

Whereas the last time (in the previous report), I looped through all of the possible weight combinations for a given race (and repeated this for a small subset of races), this time I want to loop through all of the races in the given four season period for each of the 4 models that I have, and then to compare how each model fairs over the totality of the races. In addition, I'm going to add a couple of further measures of fit to the two that I was using previously.

In [16]:
"""
Function
--------

average_from_mean : takes the output of a call to finding_percentiles and calculates
                    the mean of all of the entries in the 'diff from mean' column
                    as well as the mean of all of the absolute values of the entries in the 
                    'diff from mean' column

Parameters
----------

diff_from_mean : the 'diff from mean' column found in the output of a call to 
                 finding_percentiles

Returns
-------

average : the mean of all the entries in diff_from_mean
abs_ave : the mean of the absolute values of teh entries in diff_from_mean

Examples
--------
"""



def average_from_mean(diff_from_mean):
    
    average = np.mean(diff_from_mean)
    abs_diff = [abs(item) for item in diff_from_mean]
    abs_ave = np.mean(abs_diff)
    
    return average, abs_ave

In [17]:
"""
Function
--------

check_predictions : takes the output of a call to finding_percentiles and returns a 
                    dataframe indicating which racers fall within the middle 50 and 90
                    percents of their time distributions

Parameters
----------

df : a dataframe that is the output of a call to finding_percentiles

Returns
-------

checked_predictions : a dataframe that indicates, for each racer in a given race, whether
                      or not that racer's actual time fell within the middle 50% of their
                      predicted time distribution and whether or not the actual time fell
                      within the middle 90% of their predicted time distribution

Examples
--------
"""

def check_predictions(df):
    
    checked_predictions = []
    for i in range(len(df)):
        name = df.loc[i,'Name']
        if df.loc[i,'25th per'] <= df.loc[i,'actual time'] <= df.loc[i,'75th per']:
            middle_50 = True
        else:
            middle_50 = False

        if df.loc[i,'5th per'] <= df.loc[i,'actual time'] <= df.loc[i,'95th per']:
            middle_90 = True
        else:
            middle_90 = False
            
        checked_predictions.append([name,middle_50, middle_90])
        
    checked_predictions = pd.DataFrame(checked_predictions, columns = ['name', 'middle_50',
                                                                       'middle_90'])
    
    return checked_predictions

In [18]:
"""
Function
--------

inside_outside : takes a dataframe output by check_predictions and returns the percentage
                 of racers within the middle 50% of their distributions and the percentage
                 of racers within the middle 90% of their distributions

Parameters
----------

df : a dataframe that is the output of a call to check_predictions

Returns
-------

percent_inside_50 : a float giving the percentage of racers in a given race whose actual time
                    was in the middle 50% of their time distribution
percent_inside_90 : a float giving the percentage of racers in a given race whose actual time
                    was in the middle 90% of their time distribution

Examples
--------
"""

def inside_outside(df):
    
    inside_50 = 0
    outside_50 = 0

    for i in range(len(df)):
        if df.loc[i,'middle_50']:
            inside_50 += 1
        else:
            outside_50 += 1
         
    percent_inside_50 = inside_50/(float(inside_50) + outside_50)*100

    inside_90 = 0
    outside_90 = 0

    for i in range(len(df)):
        if df.loc[i,'middle_90']:
            inside_90 += 1
        else:
            outside_90 += 1
        
    percent_inside_90 = inside_90/(float(inside_90) + outside_90)*100

    return percent_inside_50, percent_inside_90

# Weighted season prediction

After defining a number of functions, I could begin to use them to make predictions about race times and to evaluate the predictions that were made. Before doing so, I wanted a function that would go through all of the events in a given season, make time predictions for each race, and evaluate the quality of those predictions. The function
1. ```evaluate_on_season``` runs through all of the events in a given season, using ```race_time_predictions``` to produce a dataframe of predicted times for each racer. From here, the functions defined in the above section allow us to produce and return four separate objects:
    1. median_places_correct : a dataframe containing the concatenated outputs of the medians of the results of ```evaluating_place_counts``` for the events in the given season. In other words, a dataframe which contains one column for each event in the season, whose entries are the medians of the columns of the dataframes produced by ```evaluating_place_counts```. 
    2. distance_percentages : a dataframe containing the concatenated outputs of all the results of ```evaluating_dist_from_mean``` for the events in the given season.
    3. averages_from_mean : an array containing the outputs of ```average_from_mean``` for each event in the given season.
    4. percentages_in_center : an array containing the outputs of ```inside_outside``` for each event of the season
    
Next, I ran ```evaluate_on_season``` for the last four seasons for which I had data, namely 2014-15, 2015-16, 2016-17, and 2017-18. From here, I converted the arrays returned as ```averages_from_mean``` and ```percentages_in_center``` to dataframes and used pd.concat to stitch together the four resulting dataframes (from the four seasons being modeled) of each type into a single dataframe. This left me with a total of four dataframes
- ```weight_0_averages_from_mean```
- ```weight_0_percentages_in_center```
- ```weight_0_distance_percentages```
- ```weight_0_median_places_correct```
containing information about the measures of perfomance of my model across 37 different races.

Previous Section: [Evaluation Functions](#Evaluation-Functions)

Next Section: [Probability Distribution Predictions](#Probability-Distribution-Predictions)

[Table of Contents](#Table-of-Contents)



In [19]:
"""
Function
--------

evaluate_on_season : for a given season, uses race_time_predictions to predict outcomes for
                     all events and then evaluates and returns measures of goodness of fit

Parameters
----------

season : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.
n : an integer giving the number of trials to be run for each event of the season
weight_type :  a list of lists containing the world cup and ibu similarity weightings
              for the speed prediction and prone and standing range predictions

Returns
-------
median_places_correct : a dataframe containing the concatenated output of the results of 
                        evaluating_place_counts for each event of the season
distance_percentages : a dataframe containing the concatenated output of the results of
                       evaluating_dist_from_mean for each event of the season
averages_from_mean : an array containing the outputs of average_from_mean for each event
                     of the season
percentages_in_center : an array containing the outputs of inside_outside for each event
                        of the season

Examples
--------
"""



def evaluate_on_season(season, n, weight_type):
    
    races = ['CP01','CP02','CP03','CP04','CP05','CP06','CP07','CP08','CP09','CH__','OG__']
    
    median_places_correct = pd.DataFrame()
    distance_percentages = pd.DataFrame()
    averages_from_mean = []
    percentages_in_center = []
    
    for race in races:
        code = '%(season)s:%(race)s' %{'season' : season, 'race' : race}
        print race
        try:
            predicted_times, problem_racers = race_time_predictions(season,race,n, weight_type)
        
            places = place_counts(predicted_times)
            found_percentiles = finding_percentiles(predicted_times, season, race)
            evaluated_percentiles = evaluating_percentiles(found_percentiles)
            place_order_evaluations = evaluating_place_counts(places, season, race)
            checked_percentiles = check_predictions(found_percentiles)

            median_places = pd.DataFrame(place_order_evaluations.median(), columns = [code])
            median_places_correct= pd.concat([median_places_correct,median_places], axis = 1)

            distance_percentage = evaluating_dist_from_mean(found_percentiles['diff from mean'])
            distance_percentage.columns = [code]
            distance_percentages = pd.concat([distance_percentages, distance_percentage],
                                                     axis = 1)

            race_average_from_mean = [code]
            race_average_from_mean.extend(average_from_mean(found_percentiles['diff from mean']))
            averages_from_mean.append(race_average_from_mean)
        
            percentage_in_center = [code]
            percentage_in_center.extend(inside_outside(checked_percentiles))
            percentages_in_center.append(percentage_in_center)
        
        except:  #this race doesn't exist
            pass


    return median_places_correct, distance_percentages,averages_from_mean,percentages_in_center

In [20]:
median_places_correct, distance_percentages,averages_from_mean,percentages_in_center \
= evaluate_on_season('1415',10,weight_combos[0])

CP01
CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


In [21]:
model_1415_median_places_correct, model_1415_distance_percentages,\
model_1415_averages_from_mean, model_1415_percentages_in_center \
= evaluate_on_season('1415',1000,weight_combos[0])

CP01
CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


In [22]:
model_1516_median_places_correct, model_1516_distance_percentages,\
model_1516_averages_from_mean, model_1516_percentages_in_center \
= evaluate_on_season('1516',1000,weight_combos[0])

CP01
CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


In [23]:
model_1617_median_places_correct, model_1617_distance_percentages,\
model_1617_averages_from_mean, model_1617_percentages_in_center \
= evaluate_on_season('1617',1000,weight_combos[0])

CP01
CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


In [24]:
model_1718_median_places_correct, model_1718_distance_percentages,\
model_1718_averages_from_mean, model_1718_percentages_in_center \
= evaluate_on_season('1718',1000,weight_combos[0])

CP01
CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


So what I think that I want to do next is to glue all of these things together, and then save the resulting thing to a .csv file. I will probably (I think) need to convert these things to dataframes first, but that seems fairly minor in the grand scheme of things.

In [25]:
model_1415_averages_from_mean = pd.DataFrame(model_1415_averages_from_mean)
model_1415_percentages_in_center = pd.DataFrame(model_1415_percentages_in_center)

model_1516_averages_from_mean = pd.DataFrame(model_1516_averages_from_mean)
model_1516_percentages_in_center = pd.DataFrame(model_1516_percentages_in_center)

model_1617_averages_from_mean = pd.DataFrame(model_1617_averages_from_mean)
model_1617_percentages_in_center = pd.DataFrame(model_1617_percentages_in_center)

model_1718_averages_from_mean = pd.DataFrame(model_1718_averages_from_mean)
model_1718_percentages_in_center = pd.DataFrame(model_1718_percentages_in_center)

In [26]:
weight_0_averages_from_mean = model_1415_averages_from_mean

weight_0_averages_from_mean = pd.concat([weight_0_averages_from_mean,
                                         model_1516_averages_from_mean])
weight_0_averages_from_mean = pd.concat([weight_0_averages_from_mean,
                                         model_1617_averages_from_mean])
weight_0_averages_from_mean = pd.concat([weight_0_averages_from_mean,
                                         model_1718_averages_from_mean])

#weight_0_averages_from_mean

In [27]:
weight_0_percentages_in_center = model_1415_percentages_in_center

weight_0_percentages_in_center = pd.concat([weight_0_percentages_in_center,
                                            model_1516_percentages_in_center])
weight_0_percentages_in_center = pd.concat([weight_0_percentages_in_center,
                                            model_1617_percentages_in_center])
weight_0_percentages_in_center = pd.concat([weight_0_percentages_in_center,
                                            model_1718_percentages_in_center])


In [28]:
weight_0_distance_percentages = model_1415_distance_percentages

weight_0_distance_percentages = pd.concat([weight_0_distance_percentages,
                                           model_1516_distance_percentages])
weight_0_distance_percentages = pd.concat([weight_0_distance_percentages,
                                           model_1617_distance_percentages])
weight_0_distance_percentages = pd.concat([weight_0_distance_percentages,
                                           model_1718_distance_percentages])



In [29]:
weight_0_median_places_correct = model_1415_median_places_correct

weight_0_median_places_correct = pd.concat([weight_0_median_places_correct,
                                            model_1516_median_places_correct])
weight_0_median_places_correct = pd.concat([weight_0_median_places_correct,
                                            model_1617_median_places_correct])
weight_0_median_places_correct = pd.concat([weight_0_median_places_correct,
                                            model_1718_median_places_correct])


# Probability Distribution Predictions

In order to evaluate the quality of the predictions made by my model it seemed that there were two possibilities. The first, and most obvious, was to simply compare the outcomes of my model to the actual race times. Using this method of evaluation, we might say that if a racer's time for a particular race is 1500 seconds, and it is predicted to be 1501 seconds, that the model is good, while if the racer's time is predicted to be 1500 seconds but is predicted to be 1530 seconds, the model is bad. Similarly, if a racer's most likely finish is predicted to be in the top 5, and that racer finishes third, we might say that the model is good, while if the racer finishes seventh, we might say that the model is bad. The problem here is that, given the nature of biathlon, if these are the requirements to find a _good_ model, it may well be impossible to find anything but a _bad_ model. 

Let's talk briefly about why this might be. To begin with, the shooting aspect of biathlon adds an aspect of randomness that significantly impacts overall times. On average, a penalty lap adds 25 seconds to a racer's time directly, and also (in many cases), indirectly, as a more fatigued racer is likely to be marginally slower at the end of a race. Because these shots are independent events, it is not impossible that a biathlete who normally shoots with 90% accuracy might miss 8 out of his 10 shots (for a total time penalty in the neighborhood of over 3 minutes). Similarly, a biathlete who normally shoots with 50% accuracy might make all ten shots, which might make a particular race more than 2 minutes faster than anticipated. In addition, while it seems clear that conditions affect speed, accuracy, and shooting times, it also seems clear that the condition information that we have available is not sufficient to fully explain changes in speed, etc, even in the case that we included all of the details on all of the condition variables that we have. In other words, while we might be able to find a _best_ model for these races, we will probably be unable to find a _good_ model in the sense given above.

In light of this, the best approach for evaluating the current model seems to be to compare its results to the results obtained by a naive model of the situation. That might be constructed in the following way: 
1. Observe that, for each racer, we find a wide variety of total times for the sprint race. For instance, for the last season's sprint races for Tarjei Boe, we find total times ranging from 1360 seconds to 1642 seconds, with a mean time of 1519 seconds and a standard deviation of 82.53 seconds. As a result, any naive model we choose should reflect this uncertaintly.
2. Observe that it seems reasonable to think of these values as being drawn from some sort of distribution. We can use (and did, elsewhere) use the scipy.stats module to find the best fitting probability distribution for all of the racers who competed in at least 80 races, and to choose the probability distribution that best represented the most racers. Of the nearly 90 distributions available for consideration, only four appeared more than once in the list of best models for the 34 racers with at leat 80 times. Inspection of them revealed that two were symmetric and bimodal, neither of which was an attribute that seemed to realistically model the data that I was actually seeing. The other two, <a href = "https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.exponnorm.html">stat.exponnorm</a> and <a href = "https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.johnsonsb.html">stat.johnsonsb</a> both displayed the single peaked somewhat skewed distribution that seemed to best fit the data that I was looking at. When I tried to determine which of the two models was better over the same 34 races, I found that exponnorm was better for 17, and that johnsonsb was better for 17. I decided that since exponnorm has one fewer parameter to tune than johnsonsb, and that for at least some of the racers in some of the races we would be working with relatively few prior competitions, the better choice here would be to use exponnorm as the fit.
3. Decide that a reasonable (naive) model would be to assume that the total times for each racer are drawn from an exponentially modified normal distribution, and that, further, we can get a good estimate of this by fitting an exponnorm distribution to all prior data and drawing values from this.

Given these assumptions, we build our model using the following functions:
1. ```racer_from_dist_draw```: This function takes the name of a racer and the codes (year and event) specifying a particular sprint competition, gets the race times for all previous races, fits an exponnorm distribution to them, and then randomly draws n predicted times from the distribution.
2. ```predictions_from_dist_draw```: This function calls ```racer_from_dist_draw``` for each competitor in a given race and returns a dataframe with a row for each competitor and $n$ columns, one for each predicted time (for a given competitor).
3. ```dist_draw_evaluate_on_season```: This function calls ```predictions_from_dist_draw``` for each event in the given season. It then returns four objects (which are the same as the objects returned by ```evaluate_on_season``` in weighted season prediction).
    1. median_places_correct : a dataframe containing the concatenated outputs of the medians of the results of ```evaluating_place_counts``` for the events in the given season. In other words, a dataframe which contains one column for each event in the season, whose entries are the medians of the columns of the dataframes produced by ```evaluating_place_counts```. 
    2. distance_percentages : a dataframe containing the concatenated outputs of all the results of ```evaluating_dist_from_mean``` for the events in the given season.
    3. averages_from_mean : an array containing the outputs of ```average_from_mean``` for each event in the given season.
    4. percentages_in_center : an array containing the outputs of ```inside_outside``` for each event of the season

We then ran ```dist_draw_evaluate_on_season``` for the seasons 2014-15, 2015-16, 2016-17, and 2017-18 (the most recent four seasons). From here, I converted the arrays returned as ```averages_from_mean``` and ```percentages_in_center``` to dataframes and used pd.concat to stitch together the four resulting dataframes (from the four seasons being modeled) of each type into a single dataframe. This left me with a total of four dataframes
- ```dist_draw_averages_from_mean```
- ```dist_draw_percentages_in_center```
- ```dist_draw_distance_percentages```
- ```dist_draw_median_places_correct```

containing information about the measures of perfomance of the exponentially modified normal model across 37 different races.

Previous Section: [Weighted season prediction](#Weighted-season-prediction)

Next Section: [Model comparisons](#Model-comparisons)

[Table of Contents](#Table-of-Contents)

In [30]:
"""
Function
--------

racer_from_dist_draw : predicts n race times for a given racer in a given event based on the
                       premise that the distribution of the racer's times is well modelled by
                       an exponentially modified normal function.

Parameters
----------

racer : a string containing the name of a biathlete
year : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.
event : a string that codes the event under consideration. Possible values are 'CP01', 
        'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08', 'CP09', 'CH__', 'OG__'
n : an integer giving the number of predicted times desired for the given racer.

Returns
-------

racer_predict : a list of n predicted race times

Examples
--------
"""



def racer_from_dist_draw(racer, year, event, n):
    
    col_name = ':'.join(['wc',year, event])
    racer_data = absolute_mens_time.loc[racer, :col_name]
    name = racer
    actual = float(racer_data[col_name])
    short_racer_data = racer_data[2:-1].copy()
    short_racer_data.dropna(inplace = True)
    #return short_racer_data
    predictors = len(short_racer_data)
    
    params = stats.exponnorm.fit(short_racer_data.tolist())
    
    predictions = stats.exponnorm.rvs(params[0],params[1], params[2], size = n)
    
    racer_predict = [name]#, actual]
    
    racer_predict.extend(predictions)

    return racer_predict

In [31]:
racer_from_dist_draw('FOURCADE Martin', '1617', 'CP01',10)

['FOURCADE Martin',
 1486.5137818078144,
 1688.0420575905771,
 1566.9096339352652,
 1756.3231840943986,
 1451.97440009039,
 1591.3088890365048,
 1751.9288046891268,
 1721.5022443207049,
 1546.6260766397279,
 1775.2398424322118]

In [40]:
"""
Function
--------

predictions_from_dist_draw : for a given competition, repeats predict_from_dist_draw for all
                             competitors and returns the time predictions in the form of
                             a dataframe

Parameters
----------

year : a string that codes the season under consideration. It is of the form y1y2 where
       y1 is the last two digits of the year in which the season started, and y2 is the 
       last two digits of the year in which the season ended.
event : a string that codes the event under consideration. Possible values are 'CP01', 
        'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08', 'CP09', 'CH__', 'OG__'
n : an integer giving the number of predicted times desired for the given race.

Returns
-------

predicted_times : a dataframe containing all n of the predicted times for each racer.
problem_racers : a list of racers for whome predict_from_dist_draw failed to execute. This
                 is typically due to a lack of prior races for a given competitor.

Examples
--------
"""



def predictions_from_dist_draw(year, event, n):
    
    # Get the list of racers
    
    race_code = ':'.join(['wc', year, event])
    racer_indices = absolute_mens_speed[race_code].dropna().index.tolist()

    # Make the predictions
    
    predicted_times = []
    problem_racers = []
    
    for racer in racer_indices[:-1]:
        try:
            racer_prediction = racer_from_dist_draw(racer, year, event, n)
            if str(racer_prediction[2]) != 'nan':
                predicted_times.append(racer_prediction)
            else:
                problem_racers.append(racer)
        except: # I don't think there's enough data
            pass
    
    predicted_times = pd.DataFrame(predicted_times)
    predicted_times.set_index(0, drop = True, inplace = True)
    
    return predicted_times, problem_racers

In [41]:
"""
Function
--------

dist_draw_evaluate_on_season : for a given season, uses predictions_from_dist_draw to predict
                               outcomes for all events and then evaluates and returns measures
                               of goodness of fit

Parameters
----------

season : a string that codes the season under consideration. It is of the form y1y2 where
         y1 is the last two digits of the year in which the season started, and y2 is the 
         last two digits of the year in which the season ended.
n : an integer giving the number of trials to be run for each event of the season

Returns
-------

median_places_correct : a dataframe containing the concatenated output of the results of 
                        evaluating_place_counts for each event of the season
distance_percentages : a dataframe containing the concatenated output of the results of
                       evaluating_dist_from_mean for each event of the season
averages_from_mean : an array containing the outputs of average_from_mean for each event
                     of the season
percentages_in_center : an array containing the outputs of inside_outside for each event
                        of the season

Examples
--------
"""



def dist_draw_evaluate_on_season(season, n):
    
    races = ['CP01','CP02','CP03','CP04','CP05','CP06','CP07','CP08','CP09','CH__','OG__']
    
    median_places_correct = pd.DataFrame()
    distance_percentages = pd.DataFrame()
    averages_from_mean = []
    percentages_in_center = []
    
    for race in races:
        code = '%(season)s:%(race)s' %{'season' : season, 'race' : race}
        print race
        try:
            predicted_times, problem_racers = predictions_from_dist_draw(season,race,n)
            places = place_counts(predicted_times)
            found_percentiles = finding_percentiles(predicted_times, season, race)
            evaluated_percentiles = evaluating_percentiles(found_percentiles)
            place_order_evaluations = evaluating_place_counts(places, season, race)
            checked_percentiles = check_predictions(found_percentiles)

            median_places = pd.DataFrame(place_order_evaluations.median(), columns = [code])
            median_places_correct= pd.concat([median_places_correct,median_places], axis = 1)

            distance_percentage = evaluating_dist_from_mean(found_percentiles['diff from mean'])
            distance_percentage.columns = [code]
            distance_percentages = pd.concat([distance_percentages, distance_percentage],
                                             axis = 1)

            race_average_from_mean = [code]
            race_average = average_from_mean(found_percentiles['diff from mean'])
            race_average_from_mean.extend(race_average)
            averages_from_mean.append(race_average_from_mean)
        
            percentage_in_center = [code]
            percentage_in_center.extend(inside_outside(checked_percentiles))
            percentages_in_center.append(percentage_in_center)
        
        except:  #this race doesn't exist
            pass
                
          
        
    return median_places_correct, distance_percentages,averages_from_mean,percentages_in_center

In [42]:
dist_draw_1415_median_places_correct, dist_draw_1415_distance_percentages,\
dist_draw_1415_averages_from_mean,dist_draw_1415_percentages_in_center \
= dist_draw_evaluate_on_season('1415',1000)

CP01


/anaconda2/lib/python2.7/site-packages/scipy/stats/_continuous_distns.py:1191: RuntimeWarning: divide by zero encountered in log
  return exparg + np.log(0.5 * invK * sc.erfc(-(x - invK) / np.sqrt(2)))
/anaconda2/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:2300: RuntimeWarning: Mean of empty slice.
  muhat = tmp.mean()
/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda2/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:2301: RuntimeWarning: Degrees of freedom <= 0 for slice
  mu2hat = tmp.var()
/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret /

CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


In [43]:
dist_draw_1516_median_places_correct, dist_draw_1516_distance_percentages,\
dist_draw_1516_averages_from_mean, dist_draw_1516_percentages_in_center \
= dist_draw_evaluate_on_season('1516',1000)

CP01
CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


In [44]:
dist_draw_1617_median_places_correct, dist_draw_1617_distance_percentages,\
dist_draw_1617_averages_from_mean, dist_draw_1617_percentages_in_center \
= dist_draw_evaluate_on_season('1617',1000)

CP01
CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


In [45]:
dist_draw_1718_median_places_correct, dist_draw_1718_distance_percentages,\
dist_draw_1718_averages_from_mean, dist_draw_1718_percentages_in_center \
= dist_draw_evaluate_on_season('1718',1000)

CP01
CP02
CP03
CP04
CP05
CP06
CP07
CP08
CP09
CH__
OG__


In [46]:
dist_draw_1415_averages_from_mean = pd.DataFrame(dist_draw_1415_averages_from_mean)
dist_draw_1415_percentages_in_center = pd.DataFrame(dist_draw_1415_percentages_in_center)

dist_draw_1516_averages_from_mean = pd.DataFrame(dist_draw_1516_averages_from_mean)
dist_draw_1516_percentages_in_center = pd.DataFrame(dist_draw_1516_percentages_in_center)

dist_draw_1617_averages_from_mean = pd.DataFrame(dist_draw_1617_averages_from_mean)
dist_draw_1617_percentages_in_center = pd.DataFrame(dist_draw_1617_percentages_in_center)

dist_draw_1718_averages_from_mean = pd.DataFrame(dist_draw_1718_averages_from_mean)
dist_draw_1718_percentages_in_center = pd.DataFrame(dist_draw_1718_percentages_in_center)

In [47]:
dist_draw_averages_from_mean = dist_draw_1415_averages_from_mean

dist_draw_averages_from_mean = pd.concat([dist_draw_averages_from_mean,
                                       dist_draw_1516_averages_from_mean])
dist_draw_averages_from_mean = pd.concat([dist_draw_averages_from_mean,
                                       dist_draw_1617_averages_from_mean])
dist_draw_averages_from_mean = pd.concat([dist_draw_averages_from_mean,
                                       dist_draw_1718_averages_from_mean])

In [48]:
dist_draw_percentages_in_center = dist_draw_1415_percentages_in_center

dist_draw_percentages_in_center = pd.concat([dist_draw_percentages_in_center,
                                          dist_draw_1516_percentages_in_center])
dist_draw_percentages_in_center = pd.concat([dist_draw_percentages_in_center,
                                          dist_draw_1617_percentages_in_center])
dist_draw_percentages_in_center = pd.concat([dist_draw_percentages_in_center,
                                          dist_draw_1718_percentages_in_center])

In [49]:
dist_draw_distance_percentages = dist_draw_1415_distance_percentages

dist_draw_distance_percentages = pd.concat([dist_draw_distance_percentages,
                                         dist_draw_1516_distance_percentages])
dist_draw_distance_percentages = pd.concat([dist_draw_distance_percentages,
                                         dist_draw_1617_distance_percentages])
dist_draw_distance_percentages = pd.concat([dist_draw_distance_percentages,
                                         dist_draw_1718_distance_percentages])

In [50]:
dist_draw_median_places_correct = dist_draw_1415_median_places_correct

dist_draw_median_places_correct = pd.concat([dist_draw_median_places_correct,
                                          dist_draw_1516_median_places_correct])
dist_draw_median_places_correct = pd.concat([dist_draw_median_places_correct,
                                          dist_draw_1617_median_places_correct])
dist_draw_median_places_correct = pd.concat([dist_draw_median_places_correct,
                                          dist_draw_1718_median_places_correct])

# Model comparisons

The next thing that I want to do is to make some sort of comparison for each of the four measure types that I've produced. I'd like to compare each of my three weighted models to the exponentially modified normal distribution model, and, assuming that they come out better than the exponentially modified normal distribution model, I'd like to decide which of those models seems to be the best. In order to do this, I began by defining some functions.
1. ```center_percents_compare```: This function takes two dataframes (df and df1) produced by using ```evaluate_on_season``` or ```dist_draw_evaluate_on_season``` for two different models (namely, the output ```percentages_in_center```). For each race under consideration, it determines whether df or df1 has the larger value for ```inside_50``` (in other words, whether the first model or the second has a higher percentage of racers with actual times in the middle 50% of their predicted distributions). It then counts the number of times that the first model (df) is better, the number of times that the second model (df1) is better, and the number of times that they have the same value. The function then does the same thing using the ```inside_80``` columns of the dataframes. It then returns a dataframe with the results.
2. ```averages_from_mean_compare```: This function takes two dataframes (df and df1) produced by using produced ```evaluate_on_season``` or ```dist_draw_evaluate_on_season``` for two different models (namely, the output ```average_from_mean```). For each race under consideration, it determines which model has a lower absolute value for ```average``` and which has the lower value for ```abs_ave```. These are then counted to determine in how many cases of each type the first model fairs better, in how many cases the second model fairs better, and in how many cases they fair the same. The results are then return in the form of a dataframe.
3. ```clean_up_dataframe```: This function takes a block diagonal dataframe and returns a dataframe in which all blocks have been moved to the top of the dataframe.
4. ```median_place_correct_compare```: This function takes two dataframes (df and df1) produced by using produced ```evaluate_on_season``` or ```dist_draw_evaluate_on_season``` for two different models (namely, the output ```median_places_correct```). For each race, it determines which of the two models (if either) fairs better in each of the 7 different categories. It then counts over all races under consideration and returns the results in the form of a dataframe.
5. ```median_percent_improvement```: This function takes the same two dataframes as ```median_place_correct_compare``` and divides the race categories into three groups: those where the first model performed better, those where the second model performed better, and those where the two models performed equally well. For the race categories in the first group, the function that calculates the percentage improvement of the first model over the second and computes the mean of these improvements. For the race categories in teh second group, the function calculates the percentage improvement of the second model over the first and computes the mean of these improvements. These two values are then returned.
6. ```distances_compare```: This function takes two dataframes (df and df1) produced by using produced ```evaluate_on_season``` or ```dist_draw_evaluate_on_season``` for two different models (namely, the output ```distance_percentages```). For each race, it determines which of the two models (if either) fairs better in each of the 6 different categories. It then counts over all races under consideration and returns the results in the form of a dataframe.
7. ```distance_percent_improvement```: This function takes the same two dataframes as ```distances_compare``` and divides the race categories into three groups: those where the first model performed better, those where the second model performed better, and those where the two models performed equally well. For the race categories in the first group, the function that calculates the percentage improvement of the first model over the second and computes the mean of these improvements. For the race categories in teh second group, the function calculates the percentage improvement of the second model over the first and computes the mean of these improvements. These two values are then returned.
    - N.B. There is one complication here. It is not unheard of, especially in the case of the smallest distances, for one or the other of the models to have a value of zero in their ```distance_percentages``` dataframe. This is obviously a problem, as division by zero leads to an infinite result, and therefore an infinite average. My solution has simply been to disregard these cases when finding these improvements. An obvious other choice would be to calculate straight differences rather than percent improvements.

Previous Section: [Probability Distribution Predictions](#Probability-Distribution-Predictions)

Next Section: [Complications](#Complications)

[Table of Contents](#Table-of-Contents)



In [51]:
"""
Function
--------

center_percents_compare : takes two dataframes produced by inside_outside and evaluates 
                          for how many cases each fairs better than the other

Parameters
----------

df : a dataframe produced by inside_outside via dist_draw_evaluate_on_season
df1 : a (second) dataframe produced by inside_outside via evaluate_on_season
str1 : the name of df (as a string)
str2 : the name of df1 (as a string)

Returns
-------

totals : a dataframe with counts of how frequently the first prediction method faired 
         better, how often the second faired better, and how often they tied

Examples
--------
"""



def center_percents_compare(df, df1, str1, str2):
    
    df_better = []
    df1_better = []
    same = []
    
    for i in range(len(df)):
        row = [df.iloc[i,0]]
        if df.iloc[i,1] > df1.iloc[i,1]:
            is_bigger_50 = 1
        else:
            is_bigger_50 = 0
        if df.iloc[i,2] > df1.iloc[i,2]:
            is_bigger_80 = 1
        else:
            is_bigger_80 = 0
        row.append(is_bigger_50)
        row.append(is_bigger_80)

        df_better.append(row)
        
    for i in range(len(df)):
        row = [df.iloc[i,0]]
        if df.iloc[i,1] < df1.iloc[i,1]:
            is_bigger_50 = 1
        else:
            is_bigger_50 = 0
        if df.iloc[i,2] < df1.iloc[i,2]:
            is_bigger_80 = 1
        else:
            is_bigger_80 = 0
        row.append(is_bigger_50)
        row.append(is_bigger_80)

        df1_better.append(row)

    for i in range(len(df)):
        row = [df.iloc[i,0]]
        if df.iloc[i,1] == df1.iloc[i,1]:
            is_bigger_50 = 1
        else:
            is_bigger_50 = 0
        if df.iloc[i,2] == df1.iloc[i,2]:
            is_bigger_80 = 1
        else:
            is_bigger_80 = 0
        row.append(is_bigger_50)
        row.append(is_bigger_80)

        same.append(row)

    df_betterA = zip(*df_better)
    df1_betterA = zip(*df1_better)
    sameA = zip(*same)
    totals = [[sum(df_betterA[1]),sum(df_betterA[2])],
              [sum(df1_betterA[1]),sum(df1_betterA[2])],
              [sum(sameA[1]),sum(sameA[2])]]
    
    totals = pd.DataFrame(totals, columns = ['middle 50%', 'middle 80%'], index =
                     [" ".join([str1, "is better"]), " ".join([str2, "is better"]), 'same'])
    
    return totals

In [52]:
center_percents_compare(dist_draw_percentages_in_center,weight_0_percentages_in_center,
                                                                "dist model", "my model")

,middle 50%,middle 80%
dist model is better,26,35
my model is better,11,2
same,0,0


So what I'm seeing here is more or less what I was expecting to see here. The majority of the time, the exponentially modified normal model is doing better at producing a distribution of predicted times for which the actual time is in the middle of the distribution than the weight based models are. Why might that be? One obvious theory is that the exponentially modified normal model is better than the weight based models. Another is that the spread of the exponentially modified normal model might be considerably wider than that of the weight based models. In fact, if we look at the standard deviations of the distributions produced, we find that the standard deviations of the distributions produced by the exponentially modified normal model are more than double the standard deviations produced by the weighted models. In particular, it means that the width of the middle 50% of the exponentially modified normal distribution is approximately the same as the width of the middle 80% of the weighted distributions.

Our next measure is the average distance between the actual racer times and the means of the prediction distributions. This was calculated in two different ways:
1. The average of all differences between racer time and mean of distribution for each racer.
2. The average of the absolute values of the differences.
This allows us to, among other things, determine how balanced the errors are, in addition to how large they are. Here, the closer to zero these are, the better the model is.


In [53]:
"""
Function
--------

averages_from_mean_compare : takes two dataframes produced by average_from_mean and 
                             evaluates for how many cases each fairs better than the other

Parameters
----------

df : a dataframe produced by average_from_mean via dist_draw_evaluate_on_season
df1 : a (second) dataframe produced by average_from_mean via evaluate_on_season
str1 : the name of df (as a string)
str2 : the name of df1 (as a string)

Returns
-------

totals : a dataframe with counts of how frequently the first prediction method faired 
         better, how often the second faired better, and how often they tied

Examples
--------
"""



def averages_from_mean_compare(df, df1, str1, str2):
    
    df_better = []
    df1_better = []
    same = []
    
    for i in range(len(df)):
        row = [df.iloc[i,0]]
        if df.iloc[i,1] > df1.iloc[i,1]:
            raw_average = 1
        else:
            raw_average = 0
        if df.iloc[i,2] > df1.iloc[i,2]:
            average_abs = 1
        else:
            average_abs = 0
        row.append(raw_average)
        row.append(average_abs)

        df_better.append(row)
        
    for i in range(len(df)):
        row = [df.iloc[i,0]]
        if df.iloc[i,1] < df1.iloc[i,1]:
            raw_average = 1
        else:
            raw_average = 0
        if df.iloc[i,2] < df1.iloc[i,2]:
            average_abs = 1
        else:
            average_abs = 0
        row.append(raw_average)
        row.append(average_abs)

        df1_better.append(row)

    for i in range(len(df)):
        row = [df.iloc[i,0]]
        if df.iloc[i,1] == df1.iloc[i,1]:
            raw_average = 1
        else:
            raw_average = 0
        if df.iloc[i,2] == df1.iloc[i,2]:
            average_abs = 1
        else:
            average_abs = 0
        row.append(raw_average)
        row.append(average_abs)

        same.append(row)

    df_betterA = zip(*df_better)
    df1_betterA = zip(*df1_better)
    sameA = zip(*same)
    totals = [[sum(df_betterA[1]),sum(df_betterA[2])],
              [sum(df1_betterA[1]),sum(df1_betterA[2])],
              [sum(sameA[1]),sum(sameA[2])]]
    
    totals = pd.DataFrame(totals, columns = ['average', 'absolute average'], index =
                          [" ".join([str1, "is better"]), " ".join([str2, "is better"]), 'same'])
    
    return totals

In [54]:
averages_from_mean_compare(dist_draw_averages_from_mean,weight_0_averages_from_mean,
                                                                   "dist model", "my model")

,average,absolute average
dist model is better,4,25
my model is better,33,12
same,0,0


OK, so here what we're seeing is that  our weighted model fairs better than our exponentially modified normal distribution model roughly 90% of the time in terms of average (in other words, the absolute value of the average distance between the mean of the racer's distribution and the racer's actual time was smaller for the our weighted model than for the exponentially modified normal model about 90% of the time). By contrast, we see that the exponentially modified normal model fairs better about 2/3s of the time when looking at the absolute averages, which indicates that the errors of the exponentially modified normal model are more balanced about the actual times than are those of the weighted model.

What about for the other two measures, which were the two measures that we were considering in deciding which of the weight models we should continue with? Beginning with median_places_correct, we first need to clean up the dataframe that was produced...

In [55]:
"""
Function
--------

clean_up_dataframe : takes a block diagonal dataframe and converts it to a more
                     standard form (because there's obviously something odd happening 
                     with my concatenation command)

Parameters
----------

df : a block diagonal dataframe

Returns
-------

df1 : the original dataframe with the blocks collapsed into only a single set of rows

Examples
--------
"""



def clean_up_dataframe(df):
    
    df1 = pd.DataFrame(columns = df.columns.tolist(), index = df.index.tolist())
    length = len(df)/4
    
    for i in range(df.shape[1]):
        for j in range(length):
            stuff = [df.iloc[j,i],df.iloc[j+length,i],df.iloc[j+2*length,i],
                                         df.iloc[j+3*length,i]]
            stuff = [x for x in stuff if str(x) != 'nan']
            df1.iloc[j,i] = max(stuff)
            
    df1.dropna(how = 'any', axis = 'rows', inplace = True)

    return df1

In [56]:
dist_draw_median_places_correct = clean_up_dataframe(dist_draw_median_places_correct)
weight_0_median_places_correct = clean_up_dataframe(weight_0_median_places_correct)

Now, I need to write the function that will allow me to make these comparisons. Here I want two functions:
1. a function to compare how the two models do in terms of which has more successes in each category
2. a function to compute how much better a given model fairs than another overall

In [57]:
"""
Function
--------

median_place_correct_compare : takes the results of evaluating_place_counts for two
                               different models, and returns a dataframe indicated
                               with what frequency each of the models faired best in 
                               each category

Parameters
----------

df : a dataframe produced by evaluating_place_counts via dist_draw_evaluate_on_season
df1 : a (second) dataframe produced by evaluating_place_counts via evaluate_on_season
str1 : the name of df (as a string)
str2 : the name of df1 (as a string)

Returns
-------

totals : a dataframe indicating for how many race each method faired better for each of the 
         place count ranges under consideration

Examples
--------
"""



def median_place_correct_compare(df, df1, str1, str2):
    
    df_better = []
    df1_better = []
    same = []

    for i in range(df.shape[1]):
        row = [df.columns.tolist()[i]]
        for j in range(len(df)):
            if df.iloc[j,i] > df1.iloc[j,i]:
                score = 1
            else:
                score = 0
            row.append(score)
        df_better.append(row)
        
    for i in range(df.shape[1]):
        row = [df.columns.tolist()[i]]
        for j in range(len(df)):
            if df.iloc[j,i] < df1.iloc[j,i]:
                score = 1
            else:
                score = 0
            row.append(score)
        df1_better.append(row)

    for i in range(df.shape[1]):
        row = [df.columns.tolist()[i]]
        for j in range(len(df)):
            if df.iloc[j,i] == df1.iloc[j,i]:
                score = 1
            else:
                score = 0
            row.append(score)
        same.append(row)

    #print distances_compared
        
    df_betterA = zip(*df_better)
    df1_betterA = zip(*df1_better)
    sameA = zip(*same)
    
    totals = [[sum(df_betterA[2]),sum(df_betterA[3]),sum(df_betterA[4]),
              sum(df_betterA[5]),sum(df_betterA[6]),sum(df_betterA[7]),sum(df_betterA[8])],
              [sum(df1_betterA[2]),sum(df1_betterA[3]),sum(df1_betterA[4]),
              sum(df1_betterA[5]),sum(df1_betterA[6]),sum(df1_betterA[7]),sum(df1_betterA[8])],
              [sum(sameA[2]),sum(sameA[3]),sum(sameA[4]),
              sum(sameA[5]),sum(sameA[6]),sum(sameA[7]),sum(sameA[8])]]
    
    totals = pd.DataFrame(totals, columns = df.index.tolist()[1:8],
                index = [" ".join([str1, "is better"]), " ".join([str2, "is better"]), 'same'])
    return totals

In [58]:
median_place_correct_compare(dist_draw_median_places_correct, weight_0_median_places_correct,
                                                                    "dist model", "my model")

,Correct,Within 1,Within 2,Within 3,Within 5,Within 10,Within 20
dist model is better,0,0,0,0,0,0,0
my model is better,35,36,37,37,37,37,37
same,2,1,0,0,0,0,0


So, looking at these, I can see that the exponentially modified normal distribution never fairs better than the weighted model and performs as well as the weighted model three times.  In total, the weighted model performs better than the exponentially modified normal model 256/259 = 99% of the time. This is significant, particularly since, in general, we're most interested in the order of finish for a particular race; the exact race times are something that we're less concerned with, except in the cases where some sort of record is at stake.

Another question is, when the exponentially modified normal model is better, how much better is it? When one of the weighted models is better, how much better is it? For this, I want to consider percent improvement. In other words, if the exponentially modified normal model has 5 in a category, and the weighted model has 7 in that category, the weighted model will have faired 40% beter than the exponentially modified normal model in that category. Conversely, if the exponentially modified normal model has 7 in a category and the weighted model has 5 in that category, the exponentially modified normal model will have faired 40% better than the weighted model in that category.

Once all of these percentages have been calculated, I'll calculate the averages for each group (exponentially modified normal better and weighted better). Categories where the exponentially modified normal and weighted models have the same value will simply be ignored.

In [59]:
"""
Function
--------

median_percent_improvement : takes two dataframes produced by evaluating_place_counts
                             and computes the average improvement of the first over the
                             second where the first is better, and of the second over the 
                             first where the second is better

Parameters
----------

df : a dataframe produced by evaluating_place_counts via dist_draw_evaluate_on_season
df1 : a (second) dataframe produced by evaluating_place_counts via evaluate_on_season
str1 : the name of df (as a string)
str2 : the name of df1 (as a string)

Returns
-------

df_ave_improvement : the average percentage by which the first method improves over the
                     second in cases where the first method is better
df1_ave_improvement : the average percentage by which the second method improves over the
                      first in cases where the second method is better

Examples
--------
"""



def median_percent_improvement(df, df1):
    
    df_better = []
    df1_better = []
    
    for i in range(len(df)):
        for j in range(df.shape[1]):
            if df.iloc[i,j] > df1.iloc[i,j]:
                improvement = (df.iloc[i,j]-df1.iloc[i,j])/df1.iloc[i,j]
                df_better.append(improvement)
            elif df.iloc[i,j] < df1.iloc[i,j]:
                improvement = (df1.iloc[i,j]-df.iloc[i,j])/df.iloc[i,j]
                df1_better.append(improvement)                
            else:
                pass
    
    df_ave_improvement = np.mean(df_better)*100
    df1_ave_improvement = np.mean(df1_better)*100
    return df_ave_improvement, df1_ave_improvement
    

In [60]:
median_percent_improvement(dist_draw_median_places_correct, weight_0_median_places_correct)

/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


(nan, 30.060452749115878)

So, looking at these results we see that, in the cases where the weighted model fairs the best, it is, on average, about 30% better than the exponentially modified normal model. Since there are no cases where the exponentially modified normal model fairs the best, we have no value returned there. In other words, not only does the weighted model outperform the exponentially modified normal model nearly 99% of the time, it outperforms the exponentially modified normal model on average by 30%.

The last measure that needs to be considered is that of distances. In particular, the measure that calculates for what percentage of racers in a given race is their actual time within 10 seconds of the mean, within 25 seconds of the mean, etc. Once again the dataframes were sewn together oddly, so I'll begin by fixing that.

In [61]:
weight_0_distance_percentages = clean_up_dataframe(weight_0_distance_percentages)
dist_draw_distance_percentages = clean_up_dataframe(dist_draw_distance_percentages)

In [62]:
"""
Function
--------

distances_compare : takes the results of evaluating_dist_from_mean for two
                    different models, and returns a dataframe indicated
                    with what frequency each of the models faired best in 
                     each category

Parameters
----------

df : a dataframe produced by evaluating_dist_from_mean via dist_draw_evaluate_on_season
df1 : a (second) dataframe produced by evaluating_dist_from_mean via evaluate_on_season
str1 : the name of df (as a string)
str2 : the name of df1 (as a string)

Returns
-------

totals : a dataframe indicating for how many races each method faired better for each of the 
         place count ranges under consideration

Examples
--------
"""



def distances_compare(df, df1, str1, str2):
    
    df_better = []
    df1_better = []
    same = []

    for i in range(df.shape[1]):
        row = [df.columns.tolist()[i]]
        for j in range(len(df)):
            if df.iloc[j,i] > df1.iloc[j,i]:
                score = 1
            else:
                score = 0
            row.append(score)
        df_better.append(row)
        
    for i in range(df.shape[1]):
        row = [df.columns.tolist()[i]]
        for j in range(len(df)):
            if df.iloc[j,i] < df1.iloc[j,i]:
                score = 1
            else:
                score = 0
            row.append(score)
        df1_better.append(row)

    for i in range(df.shape[1]):
        row = [df.columns.tolist()[i]]
        for j in range(len(df)):
            if df.iloc[j,i] == df1.iloc[j,i]:
                score = 1
            else:
                score = 0
            row.append(score)
        same.append(row)

    #print distances_compared
        
    df_betterA = zip(*df_better)
    df1_betterA = zip(*df1_better)
    sameA = zip(*same)
    
    totals = [[sum(df_betterA[1]),sum(df_betterA[2]),sum(df_betterA[3]),
              sum(df_betterA[4]),sum(df_betterA[5]),sum(df_betterA[6])],
              [sum(df1_betterA[1]),sum(df1_betterA[2]),sum(df1_betterA[3]),
              sum(df1_betterA[4]),sum(df1_betterA[5]),sum(df1_betterA[6])],
              [sum(sameA[1]),sum(sameA[2]),sum(sameA[3]),
              sum(sameA[4]),sum(sameA[5]),sum(sameA[6])]]
    
    totals = pd.DataFrame(totals, columns = df.index.tolist()[0:8],
              index = [" ".join([str1, "is better"]), " ".join([str2, "is better"]), 'same'])
    return totals

In [63]:
distances_compare(dist_draw_distance_percentages, weight_0_distance_percentages,
                                                                  "dist model", "my model")

,in 10,in 25,in 50,in 100,in 150,in 200
dist model is better,9,10,13,11,11,11
my model is better,23,25,24,26,26,20
same,5,2,0,0,0,6


Looking at these results, we see that my model gives better results that the exponentially modified normal model more often than not. To be somewhat more precise, my model is better in  144/222 $\approx$ 65% of cases, the exponentially modified normal normal model is better in 65/222 $\approx$ 29% of cases, and the two models are the same in 13/222 $\approx$ 6% of cases. 

Again, we want to ask the questions: When my model is better, how much better is it? When the exponentially modified normal model is better, how much better is it? This leads to the next function, ```distance_percent_improvement```.

In [64]:
"""
Function
--------

distance_percent_improvement : takes two dataframes produced by dist_draw_evaluate_on_season
                             and computes the average improvement of the first over the
                             second where the first is better, and of the second over the 
                             first where the second is better

Parameters
----------

df : a dataframe produced by evaluating_dist_from_mean via dist_draw_evaluate_on_season
df1 : a (second) dataframe produced by evaluating_dist_from_mean via evaluate_on_season

Returns
-------

df_ave_improvement : the average percentage by which the first method improves over the
                     second in cases where the first method is better
df1_ave_improvement : the average percentage by which the second method improves over the
                      first in cases where the second method is better
                      
Examples
--------
"""



def distance_percent_improvement(df, df1):
    
    df_better = []
    df1_better = []
    
    for i in range(len(df)):
        for j in range(df.shape[1]-1):
            if (df.iloc[i,j] > df1.iloc[i,j]):
                if (df1.iloc[i,j] > 0.0):
                    improvement = (df.iloc[i,j]-df1.iloc[i,j])/df1.iloc[i,j]
                    df_better.append(improvement)
            elif df.iloc[i,j] < (df1.iloc[i,j]):
                if (df.iloc[i,j] > 0.0):
                    improvement = (df1.iloc[i,j]-df.iloc[i,j])/df.iloc[i,j]
                    #print df.iloc[i,j], df1.iloc[i,j], improvement
                    df1_better.append(improvement)                
            else:
                pass
    
    df_ave_improvement = np.mean(df_better)*100
    df1_ave_improvement = np.mean(df1_better)*100
    return df_ave_improvement, df1_ave_improvement

In [65]:
distance_percent_improvement(dist_draw_distance_percentages, weight_0_distance_percentages)

(111.59012423537027, 154.46858066804899)

Looking at these, we find that, in the cases where the weighted model is better (roughly 65% of the time), they are better by 154% on average (so, if the exponentially modified normal model has 20% of racers in a particular range, the weighted models will have 51% of racers in that range), while in the cases where the exponentially modified normal models are better (roughly 29% of the time (equal values are not counted in either category)), the exponentially modified normal model is better by 112% on average. We can then calculate that, excluding cases where one of the models had a value of zero in a category and the other did not, that the weighted model is better on average by:
$$\frac{\mathbf{-52.8}*32 + 154.47*65}{100} = 83.51\%$$
which is a substantial improvement.

<u>NB</u>: To find the value $-52.8$ as the percentage improvement for my model when the exponentially modified normal model is in fact the better performer, we have
$$\begin{aligned}
\frac{a-b}{b} = 1.12 &\Rightarrow a-b = 1.12b &\\
                    &\Rightarrow a = 2.12b &\\
                    &\Rightarrow \frac{b-a}{a} &= \frac{b-2.12b}{2.12b}\\
                                           &      &= \frac{1-2.12}{2.12}\\
                                           &      &= -0.528
\end{aligned}$$
which is -52.8%.
<!-- add something about computing the -41 -->

It is perhaps worthwhile here to consider how both of these models fair versus a random assignment of racers into places in terms of ```places_correct```. Unfortunately, this is the only measure that I can realistically do this for, or I would try to do it for the other measures as well.

In [66]:
"""
Function
--------

median_random_number_correct : computes the expected median number of assignments within k 
                               places of the correct assignment for a randomly ordered list
                               of n values when the experiment is repeated m times

Parameters
----------

n : the number of racers
m : the number of trials
k : indicates the size of the range of acceptable answers

Returns
-------

expected_count : the average expected number of racers within k places of their actual times  

Examples
--------
"""

def median_random_number_correct(n, m, k):
    
    # n is the number of racers
    # m is the number of trials
    # k measure the size of the range of acceptable answers
    
    counts = []
    rate = ((2.0*k+1)/n)*m
    for j in range(int(n-2*k)):
        counts.append(rate)
    for i in range(k):
        rate = (2.0*k+1-(i+1))/n*m
        counts.append(rate)
        counts.append(rate)
        
    expected_count = np.median(counts)
    return expected_count

In [67]:
expected_medians = pd.DataFrame(columns = weight_0_median_places_correct.columns.tolist(),
                                index = weight_0_median_places_correct.index.tolist())

k_codes = {1 : 0, 2 : 1, 3 : 2, 4 : 3, 5 : 5, 6 : 10, 7 : 20}
for i in range(weight_0_median_places_correct.shape[1]):
    n = np.rint(weight_0_median_places_correct.iloc[0,i]*2)
    m = 1000
    for j in range(1,weight_0_median_places_correct.shape[0]):
        predicted = median_random_number_correct(n,m,k_codes[j])
        expected_medians.iloc[j,i] = predicted
        
expected_medians

,1415:CH__,1415:CP01,1415:CP02,1415:CP03,1415:CP04,1415:CP05,1415:CP06,1415:CP07,1415:CP08,1415:CP09,...,1617:CP09,1718:CP01,1718:CP02,1718:CP03,1718:CP04,1718:CP06,1718:CP07,1718:CP08,1718:CP09,1718:OG__
Place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Correct,7.93651,10,9.52381,9.61538,10.4167,10,9.80392,10.5263,10.2041,11.3636,...,9.43396,9.34579,9.17431,9.70874,9.61538,9.09091,10,9.80392,11.7647,11.3636
Within 1,23.8095,30,28.5714,28.8462,31.25,30,29.4118,31.5789,30.6122,34.0909,...,28.3019,28.0374,27.5229,29.1262,28.8462,27.2727,30,29.4118,35.2941,34.0909
Within 2,39.6825,50,47.619,48.0769,52.0833,50,49.0196,52.6316,51.0204,56.8182,...,47.1698,46.729,45.8716,48.5437,48.0769,45.4545,50,49.0196,58.8235,56.8182
Within 3,55.5556,70,66.6667,67.3077,72.9167,70,68.6275,73.6842,71.4286,79.5455,...,66.0377,65.4206,64.2202,67.9612,67.3077,63.6364,70,68.6275,82.3529,79.5455
Within 5,87.3016,110,104.762,105.769,114.583,110,107.843,115.789,112.245,125,...,103.774,102.804,100.917,106.796,105.769,100,110,107.843,129.412,125
Within 10,166.667,210,200,201.923,218.75,210,205.882,221.053,214.286,238.636,...,198.113,196.262,192.661,203.883,201.923,190.909,210,205.882,247.059,238.636
Within 20,325.397,410,390.476,394.231,427.083,410,401.961,431.579,418.367,465.909,...,386.792,383.178,376.147,398.058,394.231,372.727,410,401.961,482.353,465.909


In [68]:
median_place_correct_compare(weight_0_median_places_correct,expected_medians,
                                                                 "my model", "chance")

,Correct,Within 1,Within 2,Within 3,Within 5,Within 10,Within 20
my model is better,37,37,37,37,37,37,37
chance is better,0,0,0,0,0,0,0
same,0,0,0,0,0,0,0


In [69]:
median_place_correct_compare(dist_draw_median_places_correct, expected_medians,
                                                                 "dist model", "chance")

,Correct,Within 1,Within 2,Within 3,Within 5,Within 10,Within 20
dist model is better,37,37,37,37,37,37,36
chance is better,0,0,0,0,0,0,1
same,0,0,0,0,0,0,0


In [70]:
median_percent_improvement(weight_0_median_places_correct,expected_medians)

(62.715128388264283, nan)

In [71]:
median_percent_improvement(dist_draw_median_places_correct, expected_medians)

(22.742061756860394, 1.7268757443429865)

So we see that the weighted model does better than random chance here in every single category (which is perhaps not a very high barrier, but does suggest that there is some structure here), while the exponentially modified normal model does better than random chance in every category but one. In addition, the weighted model has an average improvement of 62.7% over the weighted model, which is nearly triple the 22.7% average improvement of the exponentially modified normal model.


# Complications

The most obvious complication with this model is that using it requires that we know (or at least be able to predict) something about the altitude of the race site, the snow conditions during the race, and the weather conditions during the race. While the first of these is relatively trivial to obtain by looking at one of the myriad interactive topographical world maps available on the internet, and one might reasonably be able to predict the last of them by looking at the hourly weather forecast before the beginning of the race, predicting the snow conditions poses a potential problem. The hope would be that by looking at the weather forecast, and perhaps at weather conditions for the few days prior to the event, we might be able to make some reasonable guess at what the snow conditions were likely to be like. This is certainly a shortcoming of this approach though, particularly as the snow conditions are the variable that we are using to inform our speed predictions, which account for around 80% of the total time. (And a change of 0.5 m/s in average speed over a 10000 m race produces a change in total time of around a minute and a half.)  


**Note**: While the hourly forecast is typically not available more than 24 hours before a competition, the list of racers is typically available only hours before a competition, so this does not pose a real problem.

Previous Section: [Model comparisons](#Model-comparisons)

[Table of Contents](#Table-of-Contents)
